***
# Lab 2 ML7331
> ### By Allen Hoskins, Brittany Lewandowski, Alexy Morris, Fidel Tamares


***

# Data Preparation
***

In [1]:
#import packages
import pandas as pd
import numpy as np
import lightgbm as lgb

import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,SGDClassifier,LogisticRegressionCV
from sklearn.feature_selection import SelectPercentile,f_regression
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import time
import warnings


#allen Packages
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airline_merged.csv',low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#removing unwanted column from previous export
df = df.drop(columns =['Unnamed: 0'],axis =1)

In [5]:
#dropping highly correlated columns
#https://www.codegrepper.com/code-examples/python/how+to+drop+highly+correlated+features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df = df.drop(columns = to_drop, axis=1)

In [6]:
#looking at dataframe after removing highly correlated fields.
df.head()

,year,month,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,city_origin,state_origin,country_origin,latitude_origin,longitude_origin,destination_airport,airport_destination,city_destination,state_destination,country_destination,latitude_destination,longitude_destination,scheduled_departure,departure_delay,taxi_out,scheduled_time,wheels_on,taxi_in,scheduled_arrival,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,AS,Alaska Airlines Inc.,98,N407AS,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,5,-11.0,21.0,205.0,404.0,4.0,430,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,American Airlines Inc.,2336,N3KUAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559,10,-8.0,12.0,280.0,737.0,4.0,750,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,US Airways Inc.,840,N171US,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313,20,-2.0,16.0,286.0,800.0,11.0,806,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,American Airlines Inc.,258,N3HYAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,20,-5.0,15.0,285.0,748.0,8.0,805,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,Alaska Airlines Inc.,135,N527AS,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,25,-1.0,11.0,235.0,254.0,5.0,320,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#selecting subset of data set
airports = ['BHM']

In [8]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [9]:
df_air.describe()

,year,month,day,day_of_week,flight_number,latitude_origin,longitude_origin,latitude_destination,longitude_destination,scheduled_departure,departure_delay,taxi_out,scheduled_time,wheels_on,taxi_in,scheduled_arrival,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,12514.0,12514.000000,12514.000000,12514.000000,12514.000000,1.251400e+04,1.251400e+04,12514.000000,12514.000000,12514.000000,12338.000000,12334.000000,12514.000000,12324.000000,12324.000000,12514.000000,12514.000000,12514.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000
mean,2015.0,6.150951,15.687710,3.879575,2876.476346,3.356294e+01,-8.675355e+01,34.252718,-88.175001,1192.116909,9.032906,13.783525,108.121864,1399.118306,8.632587,1416.011507,0.004075,0.014464,14.805921,0.091635,20.292293,27.732613,2.580827
std,0.0,3.384375,8.763785,1.992624,1951.952017,7.787860e-12,1.647104e-11,4.990106,8.325266,445.126945,43.869223,7.389709,36.974468,458.707388,5.844535,443.946518,0.063711,0.119397,32.820428,3.167858,73.036267,47.048389,16.190917
min,2015.0,1.000000,1.000000,1.000000,3.000000,3.356294e+01,-8.675355e+01,25.793250,-115.152330,500.000000,-20.000000,4.000000,52.000000,3.000000,1.000000,707.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000,8.000000,2.000000,1051.000000,3.356294e+01,-8.675355e+01,29.980470,-95.339720,745.000000,-5.000000,9.000000,76.000000,952.000000,5.000000,1009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000,16.000000,4.000000,2525.000000,3.356294e+01,-8.675355e+01,33.640440,-84.426940,1200.000000,-2.000000,12.000000,113.000000,1402.000000,7.000000,1414.000000,0.000000,0.000000,3.000000,0.000000,0.000000,9.000000,0.000000
75%,2015.0,9.000000,23.000000,5.000000,4592.000000,3.356294e+01,-8.675355e+01,39.175400,-83.348840,1609.000000,4.000000,16.000000,123.000000,1823.000000,10.000000,1825.000000,0.000000,0.000000,18.000000,0.000000,10.000000,34.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,6813.000000,3.356294e+01,-8.675355e+01,42.212060,-73.872610,2055.000000,1988.000000,116.000000,260.000000,2357.000000,150.000000,2325.000000,1.000000,1.000000,493.000000,144.000000,1971.000000,420.000000,240.000000


In [10]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

In [11]:
#create binary response for logistic regression
# create a function
def is_delayed(delay,axis=1):
    if delay <= 0:
        return 0
    else:
        return 1
    
# create a new column based on condition
df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87311/3939397851.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)


In [12]:
df_air.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12514 entries, 321 to 5817010
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   12514 non-null  int64  
 1   month                  12514 non-null  int64  
 2   day                    12514 non-null  int64  
 3   day_of_week            12514 non-null  int64  
 4   airline_cd             12514 non-null  object 
 5   airline_name           12514 non-null  object 
 6   flight_number          12514 non-null  int64  
 7   tail_number            12510 non-null  object 
 8   origin_airport         12514 non-null  object 
 9   airport_origin         12514 non-null  object 
 10  city_origin            12514 non-null  object 
 11  state_origin           12514 non-null  object 
 12  country_origin         12514 non-null  object 
 13  latitude_origin        12514 non-null  float64
 14  longitude_origin       12514 non-null  float64
 15

In [13]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore',drop='first')

#perform one-hot encoding on non-numeric column s
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [14]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

In [15]:
#remove target variable and column target was created from training data set
target_cols = ['is_delay','departure_delay']
feature_cols = final_df.loc[:, ~final_df.columns.isin(target_cols)].columns
X = final_df[feature_cols] # Features
y = final_df.is_delay # Target variable

In [16]:
#split original dataset into features and target variable
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [17]:
#imputing 0 into NaN values for logistic regression after OHE
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

In [18]:
#checking shape of data
print('X_train shape: ',X_train.shape,'\n')
print('X_test shape: ',X_test.shape,'\n')
print('y_train shape: ',y_train.shape,'\n')
print('y_test shape: ',y_test.shape)

X_train shape:  (10011, 1775) 

X_test shape:  (2503, 1775) 

y_train shape:  (10011,) 

y_test shape:  (2503,)


***
# Modeling and Evaluation
***

### LGBM:
***

In [19]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a
#https://lightgbm.readthedocs.io/en/latest/Parameters.html#objective

# Define a pipeline to search for the best combination of PCA truncation

pca_lgbm = PCA(n_components = .95,
               svd_solver = 'auto')


In [20]:
# Define a Standard Scaler to normalize inputs

scaler_lgbm = RobustScaler()

lr = 0.15
boost = 'dart' #'gbdt' 
obj = 'binary'
tl = 'voting' # 'data'
numiter = 1000
mbin = 300
nleaves = 42


In [21]:
# Scale -> PCA -> LR 
lgbm = lgb.LGBMClassifier(learning_rate = lr,
                          boosting = boost,
                          objective = obj,
                          tree_learner = tl,
                          #num_iterations = numiter,
                          num_leaves = nleaves,
                          max_bin = mbin,
                          extra_trees = True)


In [22]:
%%time
#start_lgbm = time.time()
pipe_lgbm = Pipeline(steps=[("scaler", scaler_lgbm),
                       ("pca", pca_lgbm),
                       ("LightGBM", lgbm)])

pipe_lgbm.fit(X_train,y_train)
#end_lgbm = time.time()

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
CPU times: user 25.8 s, sys: 3.64 s, total: 29.4 s
Wall time: 5.18 s


Pipeline(steps=[('scaler', RobustScaler()), ('pca', PCA(n_components=0.95)),
                ('LightGBM',
                 LGBMClassifier(boosting='dart', extra_trees=True,
                                learning_rate=0.15, max_bin=300, num_leaves=42,
                                objective='binary', tree_learner='voting'))])

In [23]:
#predict
y_pred_lgbm=pipe_lgbm.predict(X_test)

In [24]:
accuracy_lgbm = metrics.accuracy_score(y_test,y_pred_lgbm)
precision_lgbm = metrics.precision_score(y_test, y_pred_lgbm)
recall_lgbm = metrics.recall_score(y_test, y_pred_lgbm)

In [26]:
#print results 
print('____________Start LightGBM Base Model Results______________:')
print('Accuracy:\n',accuracy_lgbm,'\n')
print('Precision:\n',recall_lgbm,'\n')
print('Recall:\n',recall_lgbm,'\n')
print('____________End LightGBM Base Model Results________________')

____________Start LightGBM Base Model Results______________:
Accuracy:
 0.8286056731921694 

Precision:
 0.4732824427480916 

Recall:
 0.4732824427480916 

____________End LightGBM Base Model Results________________


***
## Random Forest:
***

In [27]:
#Importing required libraries.
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
#Specififying Stratified Kfold for cv.
skfcv = StratifiedKFold(n_splits=10)

In [29]:
#Creating our random forest pipeline.
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [30]:
#Fitting random forest model on training data.
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [31]:
#Predicting on our test set.
y_pred_rf = rf.predict(X_test)

In [32]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score = cross_val_score(rf, X_test, y_test, cv=skfcv, scoring='accuracy')


In [33]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_rf))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_rf))
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[1660   57]
 [ 330  456]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.83      0.97      0.90      1717
           1       0.89      0.58      0.70       786

    accuracy                           0.85      2503
   macro avg       0.86      0.77      0.80      2503
weighted avg       0.85      0.85      0.83      2503

=== All AUC Scores ===
[0.812749   0.83266932 0.84063745 0.832      0.856      0.852
 0.808      0.84       0.8        0.832     ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.830605577689243


# CatBoost

In [34]:
df_air_test = df_air.copy()

In [35]:
#convert object fields and category fields 
categories = ['year','month','day','day_of_week','airline_cd','airline_name','flight_number','origin_airport','tail_number',
              'airport_origin','city_origin','state_origin','country_origin','destination_airport','airport_destination',
              'city_destination','state_destination','country_destination','cancellation_reason']

df_air[categories] = df_air[categories].fillna('Unknown')

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87311/493185035.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air[categories] = df_air[categories].fillna('Unknown')


In [36]:
df_air[categories] = df_air[categories].astype('category')

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87311/838760448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air[categories] = df_air[categories].astype('category')


In [37]:
df_air.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12514 entries, 321 to 5817010
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   year                   12514 non-null  category
 1   month                  12514 non-null  category
 2   day                    12514 non-null  category
 3   day_of_week            12514 non-null  category
 4   airline_cd             12514 non-null  category
 5   airline_name           12514 non-null  category
 6   flight_number          12514 non-null  category
 7   tail_number            12514 non-null  category
 8   origin_airport         12514 non-null  category
 9   airport_origin         12514 non-null  category
 10  city_origin            12514 non-null  category
 11  state_origin           12514 non-null  category
 12  country_origin         12514 non-null  category
 13  latitude_origin        12514 non-null  float64 
 14  longitude_origin       12514 non-n

In [38]:
df_air['tail_number'] = df_air['tail_number'].astype('category')
df_air['airport_origin'] = df_air['airport_origin'].astype('category')

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87311/2180504395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air['tail_number'] = df_air['tail_number'].astype('category')
/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87311/2180504395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air['airport_origin'] = df_air['airport_origin'].astype('category')


In [39]:
#remove target variable and column target was created from training data set
target_cols = ['is_delay','departure_delay']
feature_cols = df_air.loc[:, ~df_air.columns.isin(target_cols)].columns
X = df_air[feature_cols] # Features
y = df_air.is_delay # Target variable

In [40]:
#split original dataset into features and target variable
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [41]:
#checking shape of data
print('X_train shape: ',X_train.shape,'\n')
print('X_test shape: ',X_test.shape,'\n')
print('y_train shape: ',y_train.shape,'\n')
print('y_test shape: ',y_test.shape)

X_train shape:  (10011, 36) 

X_test shape:  (2503, 36) 

y_train shape:  (10011,) 

y_test shape:  (2503,)


In [42]:
cat_feat = df_air.select_dtypes(include=['category']).columns.tolist()

In [50]:
%%time
#https://www.kaggle.com/code/mitribunskiy/tutorial-catboost-overview/notebook



SEED = 0
params = {'loss_function':'Logloss', # objective function
          'eval_metric':'AUC', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': SEED,
          'learning_rate': .25,
          'cat_features': cat_feat
         }
cbc = CatBoostClassifier(**params)
cbc.fit(X_train, y_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(X_test, y_test), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.7854330	best: 0.7854330 (0)	total: 14.4ms	remaining: 14.4s
200:	test: 0.8770468	best: 0.8781968 (194)	total: 2.29s	remaining: 9.08s
400:	test: 0.8909239	best: 0.8916419 (397)	total: 4.65s	remaining: 6.95s
600:	test: 0.8952986	best: 0.8953742 (557)	total: 7.08s	remaining: 4.7s
800:	test: 0.8958736	best: 0.8963160 (707)	total: 9.39s	remaining: 2.33s
999:	test: 0.8952957	best: 0.8963160 (707)	total: 11.7s	remaining: 0us

bestTest = 0.8963159899
bestIteration = 707

Shrink model to first 708 iterations.
CPU times: user 47 s, sys: 11.4 s, total: 58.4 s
Wall time: 12 s


***
# Deployment
***

***
# Exceptional Work
***